In [7]:
# Import findspark and initialize.
!pip install findspark
import findspark
findspark.init()

In [14]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,681 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Pac

In [15]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark.sql import functions as F
from pyspark import SparkFiles
import time

# Create a SparkSession
spark = SparkSession.builder.appName("HomeSales").getOrCreate()

In [16]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [17]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
home_sales = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
home_sales.show()
home_sales.createOrReplaceTempView('home_sales')

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [18]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
four_bedroom_sales = home_sales.filter(home_sales['bedrooms'] == 4)
four_bedroom_sales = four_bedroom_sales.withColumn('year', F.year('date'))
average_price_per_year = four_bedroom_sales.groupBy('year').agg(F.round(F.avg('price'), 2).alias('average_price'))
average_price_per_year.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [19]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
three_bedroom_three_bathroom_sales = home_sales.filter((home_sales['bedrooms'] == 3) & (home_sales['bathrooms'] == 3))
three_bedroom_three_bathroom_sales = three_bedroom_three_bathroom_sales.withColumn('year', F.year('date'))
average_price_per_year = three_bedroom_three_bathroom_sales.groupBy('year').agg(F.round(F.avg('price'), 2).alias('average_price'))
average_price_per_year.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2022|    292725.69|
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
+----+-------------+



In [20]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
two_floors = home_sales.filter((home_sales['bedrooms'] == 3) &
                                   (home_sales['bathrooms'] == 3) &
                                   (home_sales['floors'] == 2) &
                                   (home_sales['sqft_living'] >= 2000))
two_floors = two_floors.withColumn('year', F.year('date'))
average_price_per_year = two_floors.groupBy('year').agg(F.round(F.avg('price'), 2).alias('average_price'))
average_price_per_year.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2022|    290242.99|
|2019|    289859.14|
|2020|    292289.09|
|2021|    296330.96|
+----+-------------+



In [21]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
views = home_sales.filter(home_sales['price'] >= 350000)
average_price = views.agg(F.round(F.avg('price'), 2).alias('average_price')).collect()[0]['average_price']
if average_price >= 500000:
    view_rating = "High"
elif average_price >= 400000:
    view_rating = "Moderate"
else:
    view_rating = "Low"
print(f"The view rating for this average price is: {view_rating}")
start_time = time.time()

views.show()

end_time = time.time()
run_time = end_time - start_time
print("--- %s seconds ---" % (time.time() - start_time))

The view rating for this average price is: Moderate
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|131492a1-72e2-4a8...|2020-02-08|      2017|419199|       2|        3|       2062|    8876|     2|         0|   6|
|f1e4cef7-d151-439...|2019-0

In [22]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [23]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [24]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.841255187988281e-05 seconds ---


In [30]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy("date_built").mode("overwrite").parquet("parquet_home")

In [31]:
# 11. Read the formatted parquet data.
parquet_home_df = spark.read.parquet('parquet_home')

In [32]:
# 12. Create a temporary table for the parquet data.
parquet_home_df.createOrReplaceTempView('parquet_temp_home')

In [33]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from parquet_temp_home group by view having ROUND(AVG(price),2) <= 350000").show

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.2804586887359619 seconds ---


In [36]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [37]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False